In [1]:
import numpy as np
from collections import Counter

import scipy.io as sio
import scipy.stats as stats
from scipy.stats import nbinom

import matplotlib.pyplot as plt

### 1.Load count matrix seq-RNA

#### 1.1 Synthetic data

In [2]:
data = sio.loadmat('/home/alopez/cell_embeddings/synthetic_data/synthetic_data_tfg5.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: vi. jun. 10 11:27:14 2022',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0, 1, 1, ..., 2, 0, 0],
        [0, 1, 0, ..., 3, 3, 0],
        [0, 0, 1, ..., 4, 4, 0],
        ...,
        [0, 1, 0, ..., 1, 2, 1],
        [1, 2, 1, ..., 5, 3, 0],
        [0, 1, 1, ..., 2, 1, 0]], dtype=uint16),
 'None': MatlabOpaque([(b'names', b'MCOS', b'string', array([[3707764736],
                      [         2],
                      [         1],
                      [         1],
                      [         1],
                      [         1]], dtype=uint32))               ],
              dtype=[('s0', 'O'), ('s1', 'O'), ('s2', 'O'), ('arr', 'O')]),
 '__function_workspace__': array([[ 0,  1, 73, ...,  0,  0,  0]], dtype=uint8)}

In [3]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'None', '__function_workspace__'])

In [8]:
transposed_data = np.transpose(data["X"])
# Last column is Y
Y = transposed_data[:, -1]
# Labels: 1,-1 -> reprogrammed vs not?
# Y[Y == 0] = -1
print(f"Y dim: {Y.shape}")
# Delete Y column 
X = transposed_data[:, :-1]
print(f"X dim: {X.shape}")

In [21]:
# Transpose data -> sample x genes
X_transpose = np.transpose(X)
Y_transpose = np.transpose(Y)
print(f"Transposed X dim: {X_transpose.shape}")
print(f"Transposed Y dim: {Y_transpose.shape}")

Transposed X dim: (3999, 248)
Transposed Y dim: (248,)


#### 1.2 Real data

In [2]:
# Load data
data = sio.loadmat('chemical_reprogramming_Kimmel/datos_MSX1.mat') 
X = data['X']
Y = data['y']

print(f"X dim: {X.shape}")
print(f"Y dim: {Y.shape}. Labels: {np.unique(Y)}")
print(f"Number of samples by label: {Counter(Y.reshape(1186,))}")

X dim: (1186, 22767)
Y dim: (1186, 1). Labels: [-1  1]
Number of samples by label: Counter({-1: 773, 1: 413})


In [3]:
# Remove columns where mode is repeated more than 95% of the data
mode_values, mode_counts = stats.mode(X, axis=0)
cols_to_keep = np.sum(mode_values == X, axis=0) / X.shape[0] < 0.95
X = X[:, cols_to_keep]

print(f"X dim: {X.shape}")

X dim: (1186, 11897)


In [4]:
# Correlations: nos quedamos con los que estén entre el umbral marcado
C = np.corrcoef(X, rowvar=False)
lim_sup = 0.2
lim_inf = 0.1
# Coordenadas de los casos que lo cumplen
f, c = np.where(((C - np.diag(np.diag(C))) > lim_inf) & ((C - np.diag(np.diag(C))) < lim_sup))

In [10]:
nbinom.fit(X[:, f[i]])

AttributeError: 'nbinom_gen' object has no attribute 'fit'

In [9]:
# Vemos la distribución de cada muestra?
params_ML = []
N = X.shape[0]
i = 0
# Quito el bucle y muestro una distribución al azar
# while True:

print(f"{i+1}/{len(f)}")
print(f"{f[i]}-{c[i]}")
print("Corr:", C[f[i], c[i]])

#dist = stats.nbinom
#parmhat1 = stats.fit(dist, X[:, f[i]])
#parmhat2 = stats.fit(dist, X[:, c[i]])

datos_1 = X[:, f[i]]
parmhat1 = nbinom.fit(datos_1)
parmhat2 = nbinom.fit(X[:, c[i]])
    
#    if parmhat1[0] == np.inf or parmhat2[0] == np.inf:
#        continue
#    else:
#        params_ML.append([f[i], *parmhat1, c[i], *parmhat2, C[f[i], c[i]]])
    
plt.subplot(2, 1, 1)
data_generated1 = stats.nbinom.rvs(parmhat1[0], parmhat1[2], size=N)
plt.hist(data_generated1, density=True, alpha=0.5, label='Generated')
plt.hist(X[:, f[i]], density=True, alpha=0.5, label='Original')
plt.title(f"{f[i]}")
plt.legend()

plt.subplot(2, 1, 2)
data_generated2 = stats.nbinom.rvs(parmhat2[0], parmhat2[2], size=N)
plt.hist(data_generated2, density=True, alpha=0.5, label='Generated')
plt.hist(X[:, c[i]], density=True, alpha=0.5, label='Original')
plt.title(f"{c[i]}")
plt.legend()

plt.show()

cc = np.corrcoef(data_generated1, data_generated2)
print("Corr2:", cc[0, 1])
    
#    value = input("Press any key to continue, or 'q' to quit: ")
#    if value.lower() == 'q':
#        break
#    elif value == '':
#        if i == len(f) - 1:
#            continue
#        else:
#            i += 1

1/42809186
0-2
Corr: 0.15402853378413078


AttributeError: 'nbinom_gen' object has no attribute 'fit'